# Setting up a PEST interface from MODFLOW6 using the `PstFrom` class with `PyPestUtils` for advanced pilot point parameterization

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
import flopy

In [ ]:
import sys
sys.path.append(os.path.join("..","..","pypestutils"))

In [ ]:
import pypestutils as ppu

An existing MODFLOW6 model is in the directory `freyberg_mf6`.  Lets check it out:

In [ ]:
org_model_ws = os.path.join('freyberg_mf6')
os.listdir(org_model_ws)

You can see that all the input array and list data for this model have been written "externally" - this is key to using the `PstFrom` class. 

Let's quickly viz the model top just to remind us of what we are dealing with:

In [ ]:
id_arr = np.loadtxt(os.path.join(org_model_ws,"freyberg6.dis_idomain_layer3.txt"))
top_arr = np.loadtxt(os.path.join(org_model_ws,"freyberg6.dis_top.txt"))
top_arr[id_arr==0] = np.nan
plt.imshow(top_arr)

Now let's copy those files to a temporary location just to make sure we don't goof up those original files:

In [ ]:
tmp_model_ws = "temp_pst_from_ppu"
if os.path.exists(tmp_model_ws):
    shutil.rmtree(tmp_model_ws)
shutil.copytree(org_model_ws,tmp_model_ws)
# copy pestpp and mf6
shutil.copy(r'C:\bin\modflow\mf6.6.2_win64\bin\mf6.exe', os.path.join(tmp_model_ws,'mf6.exe'))
shutil.copy(r'C:\bin\pestpp\pestpp-ies.exe', os.path.join(tmp_model_ws,'pestpp-ies.exe'))
os.listdir(tmp_model_ws)

Now we need just a tiny bit of info about the spatial discretization of the model - this is needed to work out separation distances between parameters for build a geostatistical prior covariance matrix later.

Here we will load the flopy sim and model instance just to help us define some quantities later - flopy is not required to use the `PstFrom` class.

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=tmp_model_ws)
m = sim.get_model("freyberg6")


Here we use the simple `SpatialReference` pyemu implements to help us spatially locate parameters

In [ ]:
sr = pyemu.helpers.SpatialReference.from_namfile(
        os.path.join(tmp_model_ws, "freyberg6.nam"),
        delr=m.dis.delr.array, delc=m.dis.delc.array)
sr

Now we can instantiate a `PstFrom` class instance

In [ ]:
template_ws = "freyberg6_template"
pf = pyemu.utils.PstFrom(original_d=tmp_model_ws, new_d=template_ws,
                 remove_existing=True,
                 longnames=True, spatial_reference=sr,
                 zero_based=False,start_datetime="1-1-2018")


## Observations

So now that we have a `PstFrom` instance, but its just an empty container at this point, so we need to add some PEST interface "observations" and "parameters".  Let's start with observations using MODFLOW6 head.  These are stored in `heads.csv`:

In [ ]:
df = pd.read_csv(os.path.join(tmp_model_ws,"heads.csv"),index_col=0)
df

The main entry point for adding observations is (surprise) `PstFrom.add_observations()`.  This method works on the list-type observation output file.  We need to tell it what column is the index column (can be string if there is a header or int if no header) and then what columns contain quantities we want to monitor (e.g. "observe") in the control file - in this case we want to monitor all columns except the index column:

In [ ]:
hds_df = pf.add_observations("heads.csv",insfile="heads.csv.ins",index_cols="time",
                    use_cols=list(df.columns.values),prefix="hds",)
hds_df

We can see that it returned a dataframe with lots of useful info: the observation names that were formed (`obsnme`), the values that were read from `heads.csv` (`obsval`) and also some generic weights and group names.  At this point, no control file has been created, we have simply prepared to add this observations to the control file later.  

In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".ins")]

Nice!  We also have a PEST-style instruction file for those obs.

Now lets do the same for SFR observations:

In [ ]:
df = pd.read_csv(os.path.join(tmp_model_ws, "sfr.csv"), index_col=0)
sfr_df = pf.add_observations("sfr.csv", insfile="sfr.csv.ins", index_cols="time", use_cols=list(df.columns.values))
sfr_df

Sweet as!  Now that we have some observations, let's add parameters!

## Pilot points and `PyPestUtils`

This notebook is mostly meant to demonstrate some advanced pilot point parameterization that is possible with `PyPestUtils`, so we will only focus on HK and VK pilot point parameters.  This is just to keep the example short.  In practice, please please please parameterize boundary conditions too!

We start with simple pilot points and an isotropic variogram (bearing is irrelevant when anisotropy=1).

In [ ]:
v = pyemu.geostats.ExpVario(contribution=1.0,a=5000,bearing=0,anisotropy=1)
pp_gs = pyemu.geostats.GeoStruct(variograms=v, transform='log')

In [ ]:
pp_gs.plot()
print("spatial variogram")

Now let's get the idomain array to use as a zone array - this keeps us from setting up parameters in inactive model cells:

In [ ]:
ib = m.dis.idomain[0].array

Find HK files for the upper and lower model layers (assuming model layer 2 is a semi-confining unit)

In [ ]:
hk_arr_files = [f for f in os.listdir(tmp_model_ws) if "npf_k_" in f and f.endswith(".txt") and "layer2" not in f]
hk_arr_files

In [ ]:
arr_file = "freyberg6.npf_k_layer1.txt"
tag = arr_file.split('.')[1].replace("_","-")
pf.add_parameters(filenames=arr_file,par_type="pilotpoints",
                   par_name_base=tag,pargp=tag,zone_array=ib,
                   upper_bound=10.,lower_bound=0.1,ult_ubound=100,ult_lbound=0.01,
                   pp_options={"pp_space":3},geostruct=pp_gs, transform='log')
#let's also add the resulting hk array that modflow sees as observations
# so we can make easy plots later...
pf.add_observations(arr_file,prefix=tag,
                    obsgp=tag,zone_array=ib)

If you are familiar with how `PstFrom` has worked historically, we handed off the process to solve for the factor file (which requires solving the kriging equations for each active node) to a pure python (well, with pandas and numpy).  This was ok for toy models, but hella slow for big ugly models.  If you look at the log entries above, you should see that the instead, `PstFrom` successfully handed off the solve to `PyPestUtils`, which is exponentially faster for big models.  sweet ez! 

In [ ]:
tpl_files = [f for f in os.listdir(template_ws) if f.endswith(".tpl")]
tpl_files

In [ ]:
with open(os.path.join(template_ws,tpl_files[0]),'r') as f:
    for _ in range(2):
        print(f.readline().strip())
        


So those might look like pretty redic parameter names, but they contain heaps of metadata to help you post process things later...

So those are you standard pilot points for HK in layer 1 - same as it ever was...

## Geostatistical hyper-parameters using "conceptual points"

For the HK layer 1 pilot points, we used a standard geostatistical structure - the ever popular exponential variogram.  But what if the properties that define that variogram were themselves uncertain?  Like what if the anisotropy ellipse varied in space across the model domain?  

One way to tackle this is to assign hyper parameters at points, a tensor describing spatial correlation structure at each point. This makes sense when you have data (or "geology" = "beer and guessing") at various points to inform the correlation structure. We then use tensor interpolation to fill in the spatial structure where "beer and guessing" is missing.

The field will be calculated as:

field = interp_means + correlated_noise * 10**np.sqrt(interp_variances)

The above gives us the spatially varying correlation structure to be convolved with white noise to give the correlated_noise, but the mean and variance of the field is also uncertain (uncertain uncertainty). The field is cacluLet's start with those using the familar pilot point scheme, although they could be added for each conceptual point instead. First describe how the mean and variance are expected to change (we will keep it simple):

In [ ]:
variance_v = pyemu.geostats.ExpVario(contribution=1, a=5000)
variance_gs = pyemu.geostats.GeoStruct(variograms=variance_v)
mean_v = pyemu.geostats.ExpVario(contribution=1,a=5000)
mean_gs = pyemu.geostats.GeoStruct(variograms=mean_v)

In [ ]:
# the mean
arr_file = "freyberg6.npf_k33_layer3.txt"
tag = arr_file.split('.')[1].replace("_","-")
pf.add_parameters(filenames=arr_file,par_type="pilotpoints",
                   par_name_base=tag,pargp=tag,zone_array=ib,
                   upper_bound=10.,lower_bound=0.1,ult_ubound=100,ult_lbound=0.01,
                   pp_options={"pp_space":3},geostruct=mean_gs, transform='log')

In [ ]:
# the variance
arr_file = "freyberg6.npf_k33_layer3.txt"
tag = arr_file.split('.')[1].replace("_","-")
pf.add_parameters(filenames=arr_file,par_type="pilotpoints",
                   par_name_base=tag,pargp=tag,zone_array=ib,
                   upper_bound=10./2,lower_bound=0.1*2,ult_ubound=10,ult_lbound=0.1,
                   pp_options={"pp_space":3},geostruct=mean_gs, transform='log')

In [ ]:
#let's also add the resulting hk array that modflow sees as observations
# so we can make easy plots later...
pf.add_observations(arr_file,prefix=tag,
                    obsgp=tag,zone_array=ib)

Now we add the conceptual point hyper parameters from our friendly geologists:

In [ ]:
def ordinary_kriging(cp_coords, cp_values, grid_coords,
                     variogram_model='exponential', range_param=10000,
                     sill=1.0, nugget=0.1, background_value=0.0,
                     max_search_radius=1e20, min_points=1,
                     transform=None, min_value=1e-8):
    """
    Proper Ordinary Kriging with:
    - Variogram model selection
    - Background value fallback
    - Search radius limitation
    """
    n_grid = len(grid_coords)
    n_control = len(cp_coords)
    interp_values = np.full(n_grid, background_value)

    # Variogram function
    def variogram(h):
        if variogram_model == 'exponential':
            return nugget + (sill - nugget) * (1 - np.exp(-h / range_param))
        elif variogram_model == 'gaussian':
            return nugget + (sill - nugget) * (1 - np.exp(-(h ** 2) / (range_param ** 2)))
        else:  # spherical
            return np.where(h <= range_param,
                            nugget + (sill - nugget) * (1.5 * h / range_param - 0.5 * (h / range_param) ** 3),
                            sill)

    for i in range(n_grid):
        # Distances from this grid point to all control points
        distances = np.linalg.norm(cp_coords - grid_coords[i], axis=1)
        nearby = distances <= max_search_radius
        n_nearby = np.sum(nearby)

        if n_nearby < min_points:
            continue  # Keeps background_value

        # Build OK system: [C 1; 1^T 0] * [weights; mu] = [c; 1]
        C = np.zeros((n_nearby + 1, n_nearby + 1))
        c = np.zeros(n_nearby + 1)

        # Fill covariance matrix
        for j in range(n_nearby):
            for k in range(n_nearby):
                h = np.linalg.norm(cp_coords[nearby][j] - cp_coords[nearby][k])
                C[j, k] = variogram(h)
            # Last row/column (unbiasedness constraint)
            C[j, -1] = 1
            C[-1, j] = 1
            # Right-hand side vector
            h = distances[nearby][j]
            c[j] = variogram(h)
        c[-1] = 1  # Constraint

        # Solve
        if transform == 'log':
            if min_value is None:
                # Auto-choose based on data scale
                positive_values = cp_values[cp_values > 0]
                min_value = np.min(positive_values) * 0.01  # 1% of smallest positive value
            cp_values_transformed = np.log10(np.maximum(cp_values, min_value))
        else:
            cp_values_transformed = cp_values.copy()

        try:
            weights = solve(C, c)[:-1]
            interp_values[i] = np.sum(weights * cp_values_transformed[nearby])
        except:
            weights = np.exp(-distances[nearby] / range_param)
            interp_values[i] = np.sum(weights * cp_values_transformed[nearby]) / np.sum(weights)

    # After the loop, backtransform all at once:
    if transform == 'log':
        interp_values = 10 ** (interp_values)

    return interp_values

def generate_single_layer(control_points, interp_means, interp_variances, grid_coords_2d):
    """Generate field for a single 2D layer"""
    np.random.seed(42)
    n_grid = len(grid_coords_2d)
    cp_coords = control_points[['x', 'y']].values
    
    # Interpolate correlation tensors
    print("  Interpolating correlation tensors...")
    cp_tensors = create_2d_tensors(control_points)
    interp_tensors = interpolate_tensors_2d(cp_coords, cp_tensors, grid_coords_2d)

    # Generate correlated noise
    print("  Generating correlated noise...")
    white_noise = np.random.randn(len(grid_coords_2d))
    correlated_noise = generate_correlated_noise_2d(grid_coords_2d, interp_tensors, white_noise)

    # Combine
    field = interp_means + correlated_noise * 10**np.sqrt(interp_variances)

    return field


def generate_correlated_noise_2d(grid_coords, tensors, white_noise,
                                 n_neighbors=50, radius_multiplier=3.0,
                                 anisotropy_strength=1.0, debug=False):
    """
    Generate spatially correlated noise respecting tensor anisotropy directions.

    Parameters:
    -----------
    grid_coords : array_like, shape (n_points, 2)
        Grid coordinates [x, y]
    tensors : array_like, shape (n_points, 2, 2)
        Anisotropy tensors for each grid point
    white_noise : array_like, shape (n_points,)
        Input white noise to be correlated
    n_neighbors : int, default=10
        Number of neighbors to use for correlation
    radius_multiplier : float, default=3.0
        Multiplier for correlation radius (not used when n_neighbors is set)
    anisotropy_strength : float, default=2.0
        Strength of anisotropic correlation (higher = more directional)

    Returns:
    --------
    correlated_noise : array_like, shape (n_points,)
        Spatially correlated noise
    """
    n_points = len(grid_coords)
    correlated_noise = np.zeros(n_points)

    for i in range(n_points):
        # Diagonalize tensor to get principal axes and correlation lengths
        evals, evecs = np.linalg.eigh(tensors[i])
        if evals[1] < evals[0]:
            evals, evecs = evals[::-1], evecs[:, ::-1]

        # some debuggery
        if debug:
            print(f"Eigenvalue 0: {evals[0]}")
            print(f"Eigenvector 0: {evecs[:, 0]}")  # First COLUMN
            print(f"Eigenvalue 1: {evals[1]}")
            print(f"Eigenvector 1: {evecs[:, 1]}")  # Second COLUMN
            # Find which eigenvalue is larger
            max_idx = np.argmax(np.abs(evals))
            principal_direction = evecs[:, max_idx]
            print(f"Principal direction vector: {principal_direction}")
            angle = np.arctan2(principal_direction[1], principal_direction[0])
            angle_degrees = np.degrees(angle)
            print(f"angle: {angle_degrees}")

        major_corr_length = np.sqrt(evals[1])  # Length along major axis
        minor_corr_length = np.sqrt(evals[0])  # Length along minor axis

        # Create anisotropic transform
        # Stretch major axis to make correlation stronger in that direction
        scale_for_minor_axis = minor_corr_length / anisotropy_strength  # Compress minor
        scale_for_major_axis = major_corr_length * anisotropy_strength  # Stretch major

        transform = evecs @ np.diag([scale_for_minor_axis, scale_for_major_axis])

        # Transform all relative positions to anisotropic space
        all_dx = grid_coords - grid_coords[i]
        inv_transform = np.linalg.inv(transform)
        dx_transformed = all_dx @ inv_transform.T
        aniso_distances = np.linalg.norm(dx_transformed, axis=1)
        actual_distances = np.linalg.norm(all_dx, axis=1)

        # Select neighbors based on anisotropic distance, closer in warped space
        if n_neighbors:
            neighbor_indices = np.argsort(aniso_distances)[:min(n_neighbors, n_points)]
        else:
            max_dist = radius_multiplier * major_corr_length
            neighbor_indices = np.where(aniso_distances <= max_dist)[0]

        # Compute correlation weights using anisotropic distances
        weighted_sum = 0.0
        sum_weights = 0.0

        for j in neighbor_indices:
            # Use major correlation length as decay scale
            weight = np.exp(-(actual_distances[j]/scale_for_major_axis)**2)
            weighted_sum += weight * white_noise[j]
            sum_weights += weight

        # Normalize
        if sum_weights > 1e-10:
            correlated_noise[i] = weighted_sum / sum_weights
        else:
            correlated_noise[i] = white_noise[i]

    return correlated_noise

def create_2d_tensors(control_points):
    # Fixed tensor creation (swap major/minor in diagonal matrix)
    n = len(control_points)
    tensors = np.zeros((n, 2, 2))

    for i in range(n):
        major = control_points.iloc[i]['major']
        minor = control_points.iloc[i]['major'] / control_points.iloc[i]['ratio']
        bearing = control_points.iloc[i]['bearing'] # - 90 # assume bearing has N=0

        theta = np.radians(bearing)
        R = np.array([[np.cos(theta), -np.sin(theta)],
                      [np.sin(theta), np.cos(theta)]])

        # Fixed: Put minor first, major second
        S = np.diag([minor ** 2, major ** 2])
        tensors[i] = R @ S @ R.T

    return tensors


def interpolate_tensors_2d(cp_coords, cp_tensors, grid_coords):
    """Interpolate 2x2 tensors using log-Euclidean approach"""    
    from scipy.linalg import logm, expm, solve
    n_grid = len(grid_coords)
    interp_tensors = np.zeros((n_grid, 2, 2))

    # Convert to log space
    log_tensors = np.array([logm(t) for t in cp_tensors])

    # Interpolate each component
    for i in range(2):
        for j in range(i, 2):
            values = log_tensors[:, i, j].real
            bg_value = np.mean(values)  # Use mean of control points
            interp_values = ordinary_kriging(cp_coords, values, grid_coords, 
                                       background_value=bg_value)
            # interp_values = exponential_variogram_interpolate(cp_coords, values, grid_coords)
            # interp_values = idw_interpolate(cp_coords, values, grid_coords)
            interp_tensors[:, i, j] = interp_values
            if i != j:
                interp_tensors[:, j, i] = interp_values

    # Convert back from log space
    for i in range(n_grid):
        interp_tensors[i] = expm(interp_tensors[i])

    # # Basic ellipse visualization (most intuitive)
    # fig, ax = plt.subplots(figsize=(12, 10))
    # visualize_tensors_as_ellipses(grid_coords, interp_tensors,
    #                               subsample=20,  # Plot every 20th tensor
    #                               scale_factor=0.5,
    #                               ax=ax)

    # # Complete analysis
    # plot_tensor_field_analysis(grid_coords, interp_tensors, cp_coords)

    return interp_tensors


In [ ]:
interp_means = np.loadtxt(os.path.join(org_model_ws,arr_file)).flatten()
interp_variances = np.loadtxt(os.path.join(org_model_ws,arr_file)).flatten()/10


In [ ]:
x_centers = m.modelgrid.xcellcenters.flatten()
y_centers = m.modelgrid.ycellcenters.flatten()
grid_coords = np.column_stack([x_centers,y_centers])

In [ ]:
conceptual_points = pd.DataFrame({'x': [3, 3, 3, 3, 5, 7, 15, 20, 25, 35],
                                  'y': [3, 5, 10, 15, 5, 12, 10, 15, 3, 10],
                                  'bearing': [-40, -30, -20, -20, -30, -40, 0, 0, 10 ,200],
                                  'major': [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 2000],
                                  'ratio': [5, 5, 2, 2, 2, 2, 2, 5, 5, 2]})

In [ ]:
conceptual_points

In [ ]:
field = generate_single_layer(conceptual_points, interp_means, interp_variances, grid_coords)

In [ ]:
plt.imshow(field.reshape(40,20))
plt.colorbar()

### build the control file, pest interface files, and forward run script
At this point, we have some parameters and some observations, so we can create a control file:

In [ ]:
pf.mod_sys_cmds.append("mf6")
pf.pre_py_cmds.insert(0,"import sys")
pf.pre_py_cmds.insert(1,"sys.path.append(os.path.join('..','..','..','pypestutils'))")
pst = pf.build_pst()

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(template_ws,"forward_run.py"))]

## Setting initial parameter bounds and values

Here is some gory detail regarding defining the hyper parameters for both layer 3 HK and layer 2 VK...

In [ ]:
#set the initial and bounds for the fill values
par = pst.parameter_data

apar = par.loc[par.pname.str.contains("aniso"),:]
bpar = par.loc[par.pname.str.contains("bearing"), :]
par.loc[apar.parnme.str.contains("layer3").index,"parval1"] = 3
par.loc[apar.parnme.str.contains("layer3").index,"parlbnd"] = 1
par.loc[apar.parnme.str.contains("layer3").index,"parubnd"] = 5

par.loc[apar.parnme.str.contains("layer2").index,"parval1"] = 2
par.loc[apar.parnme.str.contains("layer2").index,"parlbnd"] = 0
par.loc[apar.parnme.str.contains("layer2").index,"parubnd"] = 4

par.loc[bpar.parnme.str.contains("layer3").index,"parval1"] = 0
par.loc[bpar.parnme.str.contains("layer3").index,"parlbnd"] = -20
par.loc[bpar.parnme.str.contains("layer3").index,"parubnd"] = 20

par.loc[bpar.parnme.str.contains("layer2").index,"parval1"] = 0
par.loc[bpar.parnme.str.contains("layer2").index,"parlbnd"] = 20
par.loc[bpar.parnme.str.contains("layer2").index,"parubnd"] = 40

cat1par = par.loc[par.apply(lambda x: x.threshcat=="0" and x.usecol=="threshfill",axis=1),"parnme"]
cat2par = par.loc[par.apply(lambda x: x.threshcat == "1" and x.usecol == "threshfill", axis=1), "parnme"]
assert cat1par.shape[0] == 1
assert cat2par.shape[0] == 1

cat1parvk = [p for p in cat1par if "k:1" in p]
cat2parvk = [p for p in cat2par if "k:1" in p]
for lst in [cat2parvk,cat1parvk]:
    assert len(lst) > 0

#these are the values that will fill the two categories of VK - 
# one is low (clay) and one is high (sand - the windows)
par.loc[cat1parvk, "parval1"] = 0.0001
par.loc[cat1parvk, "parubnd"] = 0.01
par.loc[cat1parvk, "parlbnd"] = 0.000001
par.loc[cat1parvk, "partrans"] = "log"
par.loc[cat2parvk, "parval1"] = 0.1
par.loc[cat2parvk, "parubnd"] = 1
par.loc[cat2parvk, "parlbnd"] = 0.01
par.loc[cat2parvk, "partrans"] = "log"


cat1par = par.loc[par.apply(lambda x: x.threshcat == "0" and x.usecol == "threshproportion", axis=1), "parnme"]
cat2par = par.loc[par.apply(lambda x: x.threshcat == "1" and x.usecol == "threshproportion", axis=1), "parnme"]

assert cat1par.shape[0] == 1
assert cat2par.shape[0] == 1

#these are the proportions of clay and sand in the resulting categorical array
#really under the hood, only the first one is used, so we can fix the other.
par.loc[cat1par, "parval1"] = 0.95
par.loc[cat1par, "parubnd"] = 1.0
par.loc[cat1par, "parlbnd"] = 0.9
par.loc[cat1par,"partrans"] = "none"

# since the apply method only looks that first proportion, we can just fix this one
par.loc[cat2par, "parval1"] = 1
par.loc[cat2par, "parubnd"] = 1
par.loc[cat2par, "parlbnd"] = 1
par.loc[cat2par,"partrans"] = "fixed"


# Generating a prior parameter ensemble, then run and viz a real

In [ ]:
np.random.seed(122346)
pe = pf.draw(num_reals=100)

In [ ]:
pe.to_csv(os.path.join(template_ws,"prior.csv"))

In [ ]:
real = 0
pst_name = "real_{0}.pst".format(real)
pst.parameter_data.loc[pst.adj_par_names,"parval1"] = pe.loc[real,pst.adj_par_names].values

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d,pst_name))

In [ ]:
pf.new_d

In [ ]:
pst_name

In [ ]:
pyemu.os_utils.run("pestpp-ies {0}".format(pst_name),cwd=pf.new_d)

In [ ]:
pst.set_res(os.path.join(pf.new_d,pst_name.replace(".pst",".base.rei")))
res = pst.res
obs = pst.observation_data
grps = [o for o in obs.obgnme.unique() if o.startswith("npf") and "result" not in o and "aniso" not in o]
grps

In [ ]:
gobs = obs.loc[obs.obgnme.isin(grps),:].copy()
gobs["i"] = gobs.i.astype(int)
gobs["j"] = gobs.j.astype(int)
gobs["k"] = gobs.obgnme.apply(lambda x: int(x.split('-')[2].replace("layer","")) - 1)

In [ ]:
uk = gobs.k.unique()
uk.sort()

In [ ]:
for k in uk:
    kobs = gobs.loc[gobs.k==k,:]
    ug = kobs.obgnme.unique()
    ug.sort()
    fig,axes = plt.subplots(1,4,figsize=(20,6))
    axes = np.atleast_1d(axes)
    for ax in axes:
        ax.set_frame_on(False)
        ax.set_yticks([])
        ax.set_xticks([])
    for g,ax in zip(ug,axes):
        gkobs = kobs.loc[kobs.obgnme==g,:]
        
        arr = np.zeros_like(top_arr)
        arr[gkobs.i,gkobs.j] = res.loc[gkobs.obsnme,"modelled"].values
        ax.set_aspect("equal")
        label = ""
        if "bearing" not in g and "aniso" not in g:
            arr = np.log10(arr)
            label = "$log_{10}$"
        cb = ax.imshow(arr)
        plt.colorbar(cb,ax=ax,label=label)
        ax.set_title("layer: {0} group: {1}".format(k+1,g),loc="left",fontsize=15)
        
    plt.tight_layout()
    plt.show()
    plt.close(fig)

Stunning isn't it?!  There is clearly a lot subjectivity in the form of defining the prior for the hyper parameters required to use these non-stationary geostats, but they do afford more opportunities to express (stochastic) expert knowledge.  To be honest, there was a lot of experimenting with this notebook to get these figures to look this way - playing with variograms and parameter initial values and bounds a lot.  You encouraged to do the same!  scroll back up, change things, and "restart kernel and run all" - this will help build some better intution, promise....